## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')
#Ignore future version warning (version warning)

In [2]:
#import modules
import sklearn as sk
import seaborn as sns
import numpy as np
import keras_tuner as kt
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.svm import SVR, LinearSVR
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

# REGRESSION

## Data Preprocessing & Feature selection

In [3]:
def get_best_n(df_input,model):
    scaler = StandardScaler()
    best = 0
    n_pca = 0
    scaler.fit(df_input)
    scaled_data = scaler.transform(df_input)
    for i, val in enumerate(range(len(scaled_data[1]))):
        if i == 0:
            i = 1
        pca = PCA(n_components = i)
        pca.fit(scaled_data)
        x_pca = pca.transform(scaled_data)
        nX_train, nX_test, ny_train, ny_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=123)
        n_model = model
        n_model = n_model.fit(nX_train, ny_train)
        val = n_model.score(nX_test,ny_test)
        if abs(val) > abs(best):
            best = val
            n_pca = i
    return (n_pca,best)

In [4]:
def best_pre_process(data,model,to_drop):
    proc_1 = get_best_n(data,model)
    val_1 = proc_1[1]
    df_proc_2 = data.drop(to_drop,axis=1)
    proc_2 = get_best_n(df_proc_2,model)
    val_2 = proc_2[1]
    if val_1 > val_2:
        return (proc_1[0],"Only PCA")
    else:
        return (proc_2[0],"Pairwise & PCA")

In [5]:
df_regressionData = pd.read_excel("trainDataset.xls", usecols = ['RelapseFreeSurvival (outcome)','Age','original_shape_Elongation','original_shape_Flatness','original_shape_LeastAxisLength','original_shape_MajorAxisLength','original_shape_Maximum2DDiameterColumn','original_shape_Maximum2DDiameterRow','original_shape_Maximum2DDiameterSlice','original_shape_Maximum3DDiameter','original_shape_MeshVolume','original_shape_MinorAxisLength','original_shape_Sphericity','original_shape_SurfaceArea','original_shape_SurfaceVolumeRatio','original_shape_VoxelVolume','original_firstorder_10Percentile','original_firstorder_90Percentile','original_firstorder_Energy','original_firstorder_Entropy','original_firstorder_InterquartileRange','original_firstorder_Kurtosis','original_firstorder_Maximum','original_firstorder_MeanAbsoluteDeviation','original_firstorder_Mean','original_firstorder_Median','original_firstorder_Minimum','original_firstorder_Range','original_firstorder_RobustMeanAbsoluteDeviation','original_firstorder_RootMeanSquared','original_firstorder_Skewness','original_firstorder_TotalEnergy','original_firstorder_Uniformity','original_firstorder_Variance','original_glcm_Autocorrelation','original_glcm_ClusterProminence','original_glcm_ClusterShade','original_glcm_ClusterTendency','original_glcm_Contrast','original_glcm_Correlation','original_glcm_DifferenceAverage','original_glcm_DifferenceEntropy','original_glcm_DifferenceVariance','original_glcm_Id','original_glcm_Idm','original_glcm_Idmn','original_glcm_Idn','original_glcm_Imc1','original_glcm_Imc2','original_glcm_InverseVariance','original_glcm_JointAverage','original_glcm_JointEnergy','original_glcm_JointEntropy','original_glcm_MCC','original_glcm_MaximumProbability','original_glcm_SumAverage','original_glcm_SumEntropy','original_glcm_SumSquares','original_gldm_DependenceEntropy','original_gldm_DependenceNonUniformity','original_gldm_DependenceNonUniformityNormalized','original_gldm_DependenceVariance','original_gldm_GrayLevelNonUniformity','original_gldm_GrayLevelVariance','original_gldm_HighGrayLevelEmphasis','original_gldm_LargeDependenceEmphasis','original_gldm_LargeDependenceHighGrayLevelEmphasis','original_gldm_LargeDependenceLowGrayLevelEmphasis','original_gldm_LowGrayLevelEmphasis','original_gldm_SmallDependenceEmphasis','original_gldm_SmallDependenceHighGrayLevelEmphasis','original_gldm_SmallDependenceLowGrayLevelEmphasis','original_glrlm_GrayLevelNonUniformity','original_glrlm_GrayLevelNonUniformityNormalized','original_glrlm_GrayLevelVariance','original_glrlm_HighGrayLevelRunEmphasis','original_glrlm_LongRunEmphasis','original_glrlm_LongRunHighGrayLevelEmphasis','original_glrlm_LongRunLowGrayLevelEmphasis','original_glrlm_LowGrayLevelRunEmphasis','original_glrlm_RunEntropy','original_glrlm_RunLengthNonUniformity','original_glrlm_RunLengthNonUniformityNormalized','original_glrlm_RunPercentage','original_glrlm_RunVariance','original_glrlm_ShortRunEmphasis','original_glrlm_ShortRunHighGrayLevelEmphasis','original_glrlm_ShortRunLowGrayLevelEmphasis','original_glszm_GrayLevelNonUniformity','original_glszm_GrayLevelNonUniformityNormalized','original_glszm_GrayLevelVariance','original_glszm_HighGrayLevelZoneEmphasis','original_glszm_LargeAreaEmphasis','original_glszm_LargeAreaHighGrayLevelEmphasis','original_glszm_LargeAreaLowGrayLevelEmphasis','original_glszm_LowGrayLevelZoneEmphasis','original_glszm_SizeZoneNonUniformity','original_glszm_SizeZoneNonUniformityNormalized','original_glszm_SmallAreaEmphasis','original_glszm_SmallAreaHighGrayLevelEmphasis','original_glszm_SmallAreaLowGrayLevelEmphasis','original_glszm_ZoneEntropy','original_glszm_ZonePercentage','original_glszm_ZoneVariance','original_ngtdm_Busyness','original_ngtdm_Coarseness','original_ngtdm_Complexity','original_ngtdm_Contrast','original_ngtdm_Strength'])
df_regressionData = df_regressionData.replace(999, np.nan)

In [6]:
df_regressionData.dtypes.unique()

array([dtype('float64'), dtype('int64')], dtype=object)

In [7]:
df_regressionData.shape

(400, 109)

In [8]:
df_regressionData.isna().sum().sum()

0

In [9]:
df_result = df_regressionData.iloc[:, 0]

In [10]:
scaler = MinMaxScaler()
df_regressionData = pd.DataFrame(scaler.fit_transform(df_regressionData),columns=df_regressionData.columns,index=df_regressionData.index)

In [11]:
df_input = df_regressionData.iloc[:, 1:]

In [12]:
to_drop = ['original_shape_Maximum2DDiameterSlice',
            'original_shape_SurfaceArea',
            'original_firstorder_Energy',
            'original_firstorder_Mean',
            'original_firstorder_MeanAbsoluteDeviation',
            'original_firstorder_Uniformity',
            'original_firstorder_Entropy',
            'original_firstorder_Median', 
            'original_shape_VoxelVolume',
            'original_glcm_ClusterProminence',
            'original_glcm_ClusterTendency',
            'original_firstorder_Variance',
            'original_glcm_Contrast',
            'original_glcm_DifferenceAverage',
            'original_glcm_DifferenceVariance',
            'original_glcm_Id',
            'original_glcm_Idm',
            'original_glcm_Idmn',
            'original_glcm_DifferenceEntropy',
            'original_glcm_Idn',
            'original_glcm_JointEnergy',
            'original_glcm_JointEntropy',
            'original_glcm_SumEntropy',
            'original_glcm_SumSquares',
            'original_glcm_MaximumProbability',
            'original_gldm_LowGrayLevelEmphasis',
            'original_glrlm_GrayLevelNonUniformityNormalized',
            'original_gldm_HighGrayLevelEmphasis',
            'original_glrlm_GrayLevelNonUniformity',
            'original_glrlm_LongRunHighGrayLevelEmphasis',
            'original_gldm_GrayLevelNonUniformity',
            'original_gldm_GrayLevelVariance',
            'original_gldm_LargeDependenceEmphasis',
            'original_gldm_LargeDependenceHighGrayLevelEmphasis',
            'original_gldm_LargeDependenceLowGrayLevelEmphasis',
            'original_glrlm_GrayLevelVariance',
            'original_glszm_LowGrayLevelZoneEmphasis',
            'original_glrlm_ShortRunEmphasis',
            'original_glszm_SmallAreaLowGrayLevelEmphasis',
            'original_glrlm_LongRunEmphasis',
            'original_glszm_LargeAreaHighGrayLevelEmphasis',
            'original_ngtdm_Complexity',
            'original_ngtdm_Coarseness',
            'original_ngtdm_Contrast',
            'original_glcm_SumAverage'
            ]

In [13]:
df_corr_drop = df_input.drop(to_drop,axis=1)

## Linear Regression

In [14]:
result = best_pre_process(df_input,LinearRegression(),to_drop)
result

(38, 'Pairwise & PCA')

In [15]:
pca = PCA(n_components = result[0])
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [16]:
gs_linreg = GridSearchCV(LinearRegression(),{
    'fit_intercept': [False,True], #Default = False
    'n_jobs': [0,1,10,50],
}, cv=10, return_train_score=False, scoring = 'neg_mean_absolute_error')
gs_linreg.fit(x_pca, df_result)

GridSearchCV(cv=10, estimator=LinearRegression(),
             param_grid={'fit_intercept': [False, True],
                         'n_jobs': [0, 1, 10, 50]},
             scoring='neg_mean_absolute_error')

In [17]:
dfgs = pd.DataFrame(gs_linreg.cv_results_)
dfgs[['param_fit_intercept','param_n_jobs', 'mean_test_score']]

,param_fit_intercept,param_n_jobs,mean_test_score
0,False,0,-78.909205
1,False,1,-78.909205
2,False,10,-78.909205
3,False,50,-78.909205
4,True,0,-23.680259
5,True,1,-23.680259
6,True,10,-23.680259
7,True,50,-23.680259


## Support Vector Machines

In [103]:
result = best_pre_process(df_input,SVR(),to_drop)
result

(8, 'Pairwise & PCA')

In [104]:
pca = PCA(n_components = 8)
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [105]:
gs_SVM = GridSearchCV(SVR(),{
    'kernel' : ['sigmoid'], 
    'gamma' : ['auto'],
    'coef0' : [1],
}, cv=10, return_train_score=False, scoring = 'neg_mean_absolute_error')
gs_SVM.fit(x_pca,df_result)

GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'coef0': [1], 'gamma': ['auto'],
                         'kernel': ['sigmoid']},
             scoring='neg_mean_absolute_error')

In [106]:
pd.set_option('display.max_rows', None)

In [107]:
dfgs = pd.DataFrame(gs_SVM.cv_results_)
dfgs[['param_kernel', 'param_gamma','param_coef0', 'mean_test_score']]

,param_kernel,param_gamma,param_coef0,mean_test_score
0,sigmoid,auto,1,-21.682506


## Decision Trees

In [ ]:
result = best_pre_process(df_input,DecisionTreeRegressor(),to_drop)
result

In [ ]:
#Peforming this PCA yields worse results (do not use)

# pca = PCA(n_components = result[0])
# if result[1] == "Pairwise & PCA":
#     pca.fit(df_corr_drop)
#     x_pca = pca.transform(df_corr_drop)
# else:
#     pca.fit(df_input)
#     x_pca = pca.transform(df_input)

In [ ]:
gs_DTR = GridSearchCV(DecisionTreeRegressor(),{
    'criterion' : ['squared_error','friedman_mse','absolute_error','poisson'],
    'splitter' : ['best','random'], 
}, cv=10, return_train_score=False, scoring = 'neg_mean_absolute_error')
gs_DTR.fit(df_input,df_result)

In [ ]:
dfgs = pd.DataFrame(gs_DTR.cv_results_)
dfgs[['param_criterion', 'param_splitter', 'mean_test_score']]

##  Multi-Layer Perceptron Neural Network.

In [ ]:
result = best_pre_process(df_input,MLPRegressor(),to_drop)
result

In [ ]:
pca = PCA(n_components = result[0])
if result[1] == "Pairwise & PCA":
    pca.fit(df_corr_drop)
    x_pca = pca.transform(df_corr_drop)
else:
    pca.fit(df_input)
    x_pca = pca.transform(df_input)

In [ ]:
xR_train, xR_test, yR_train, yR_test = train_test_split(x_pca,df_result, test_size=0.3, random_state=1)

In [ ]:
#Eval on hyper-parameters of MLPRegressor
gs_MLP = GridSearchCV(MLPRegressor(),{
    'activation': ['identity', 'logistic', 'tanh', 'relu'], #default =relu
    'solver': ['lbfgs','sgd','adam'] #default =adam
}, cv=5, return_train_score=False, scoring = 'neg_mean_absolute_error')
gs_MLP.fit(x_pca,df_result)

In [ ]:
dfgs = pd.DataFrame(gs_MLP.cv_results_)
dfgs[['param_activation', 'param_solver', 'mean_test_score']]

In [ ]:
gs_MLP2 = GridSearchCV( MLPRegressor(),{
    'solver': ['adam'],
    'max_fun':[15000,20000,10000,25000] #default is 15000 and only applys to lbfgs
}, cv=5, return_train_score=False,scoring = 'neg_mean_absolute_error' )
gs_MLP2.fit(x_pca,df_result)

In [ ]:
dfgs = pd.DataFrame(gs_MLP2.cv_results_)
dfgs[['param_solver', 'param_max_fun', 'mean_test_score']]

# CLASSIFICATION

## Data Handling

In [ ]:
df_raw_data = pd.read_excel('trainDataset.xls')
df_raw_data = df_raw_data.drop(['RelapseFreeSurvival (outcome)', 'ID'], axis = 1)

df_raw = df_raw_data.replace(999, np.nan)
df_raw['TrippleNegative'] = df_raw['TrippleNegative'].fillna(df_raw['TrippleNegative'].mode()[0])
df_raw['ChemoGrade'] = df_raw['ChemoGrade'].fillna(df_raw['ChemoGrade'].mode()[0])
df_raw['Proliferation'] = df_raw['Proliferation'].fillna(df_raw['Proliferation'].mode()[0])
df_raw['HistologyType'] = df_raw['HistologyType'].fillna(df_raw['HistologyType'].mode()[0])
df_raw['LNStatus'] = df_raw['LNStatus'].fillna(df_raw['LNStatus'].mode()[0])
df_raw['TumourStage'] = df_raw['TumourStage'].fillna(df_raw['TumourStage'].mode()[0])
df_raw = df_raw.dropna()

target = df_raw['pCR (outcome)']
learning = df_raw.drop('pCR (outcome)', axis = 1)
cols = list(learning.columns)
scaler = StandardScaler()
scaler.fit(learning)
scaled = scaler.transform(learning)
data = pd.DataFrame(scaled, columns=cols)

pca = PCA(.95)
pca.fit(data)
reduced = pca.transform(data)

dfClass = pd.DataFrame(reduced)

In [ ]:
xC_train = dfClass.to_numpy()
yC_train = target

## Logistic Regression

In [ ]:
gs_logreg = GridSearchCV(LogisticRegression(solver = 'liblinear'),{
    'penalty': ['l1','l2'] #default = l2
}, cv=5, return_train_score=False, scoring = 'accuracy')
gs_logreg.fit(xC_train, yC_train)

In [ ]:
dfgs = pd.DataFrame(gs_logreg.cv_results_)
dfgs[['param_penalty', 'mean_test_score']]

## Support Vector Machines

In [ ]:
gs_SVC = GridSearchCV(SVC(),{
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], #default rbf
    'degree' : [2,3], #default = 3, only applys to kernel: poly
}, cv=10, return_train_score=False, scoring = 'accuracy')
gs_SVC.fit(xC_train,yC_train)

In [ ]:
dfgs = pd.DataFrame(gs_SVC.cv_results_)
dfgs[['param_kernel','param_degree', 'mean_test_score']]
#Having 'kernel : poly' with a degree of 2 improves the accuracy 

In [ ]:
#Eval on hyper-parameters of LinearRegression
gs_LinearSVC = GridSearchCV(LinearSVC(),{
    'tol' : [0.00001, 0.0001, 0.001], 
    'loss' : ['epsilon_insensitive', 'squared_epsilon_insensitive'] 
}, cv=10, return_train_score=False, scoring = 'accuracy')
gs_LinearSVC.fit(xC_train,yC_train)

In [ ]:
dfgs = pd.DataFrame(gs_LinearSVC.cv_results_)
dfgs[['param_loss', 'param_tol', 'mean_test_score']]
#Having 'kernel : poly' with a degree of 2 improves the accuracy 

## Decision Trees

In [ ]:
gs_RFC = GridSearchCV(RandomForestClassifier(),{
    'criterion' : ['gini','entropy'],
    'max_features' : ['sqrt','log2']
}, cv=5, return_train_score=False, scoring = 'accuracy')
gs_RFC.fit(xC_train,yC_train)

In [ ]:
dfgs = pd.DataFrame(gs_RFC.cv_results_)
dfgs[['param_criterion','param_max_features','mean_test_score']]

## Multi-Layer Perceptron Neural Network

In [ ]:
mlpc = MLPClassifier(max_iter=2500, alpha=0.00001, hidden_layer_sizes=12, random_state=3)
mlpc.fit(xC_train, yC_train)

In [ ]:
gs_MLPC = GridSearchCV(MLPClassifier(),{
    'activation': ['identity','logistic','tanh','relu']
}, cv=5, return_train_score=False, scoring = 'accuracy')
gs_MLPC.fit(xC_train,yC_train)

In [ ]:
dfgs = pd.DataFrame(gs_MLPC.cv_results_)
dfgs[['param_activation','mean_test_score']]

In [ ]:
features = list(X)
pca = PCA(n_components=10)
Xs_pca = pca.fit_transform(X)
Xs_pca=Xs_pca[:,0:5] #retain the first 5 PC
y.head()

def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(5, input_dim=5,activation="relu"))    
    
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu")
            )
        
        # Tune dropout layer with values from 0 - 0.3 with stepsize of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.3, step=0.1)))
    
    # Add output layer.
    model.add(keras.layers.Dense(units=10, activation="softmax"))
    
    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    
    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])
    model.summary()
    return model
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)



tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')




tuner.search(X_train, y_train, epochs=5, validation_split=0.2, callbacks=[stop_early], verbose=2)



best_hps=tuner.get_best_hyperparameters()[0]
h_model = tuner.hypermodel.build(best_hps)
h_model.fit(X_train, y_train, epochs=25, validation_split=0.2, callbacks=[stop_early], verbose=2)



h_model.evaluate(X_test, y_test, return_dict=True)